In [1]:
# Set up imports
import pandas as pd
import numpy as np
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import sys

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\chave\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Load datasets
ohio_df = pd.read_csv(r"Unstandardized Data\Ohio_completed.csv")
ma_df = pd.read_csv(r"Unstandardized Data\MA_completed.csv")

In [3]:
# Look and check through Ohio Data
ohio_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Legal,Credential,Name,"City, State",Action,Date,Filename,Unnamed: 7,Text
0,0,0,0,M2013-228,RM,"UNITED MORTGAGE GROUP, INC.","PARMA, OH",REPORT AND RECOMMENDATION,2000-01-27,NaN,NaN,NaN
1,1,1,1,M2013-227,RM,"SUPERIOR HOME MORTGAGE, LLC","EUCLID, OH",NOTICE OF INTENT TO DENY,2000-04-12,NaN,NaN,NaN
2,2,2,2,M2013-228,RM,"UNITED MORTGAGE GROUP, INC.","PARMA, OH",DIVISION ORDER,2000-04-12,NaN,NaN,NaN
3,3,3,3,M2013-227,RM,"SUPERIOR HOME MORTGAGE, LLC","EUCLID, OH",REPORT AND RECOMMENDATION,2000-11-02,NaN,NaN,NaN
4,4,4,4,M2013-221,RM,"T. P. O. T. T., INC.","COLUMBUS, OH",NOTICE OF INTENT TO REVOKE,2001-04-18,_00_18_24_00182476,NaN,STATE OF OHIO\nDEPARTMENT OF COMMERCE\nDIVISIO...


In [4]:
# Delete extra columns
ohio_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1','Unnamed: 7', 'Filename','Legal'], axis=1, inplace=True)

In [5]:
# Look and check through MA Data
ma_df.head()
ma_df.dtypes

Unnamed: 0              int64
Unnamed: 0.1            int64
Unnamed: 0.1.1          int64
Unnamed: 0.1.1.1        int64
Date                   object
Year                  float64
Enforcement Action     object
Entity Name            object
Entity Type            object
Conduct                object
Location               object
URL                    object
Filename               object
Text                   object
dtype: object

In [6]:
# Delete extra columns
ma_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1','URL', 'Filename','Conduct'], axis=1, inplace=True)

In [7]:
ma_df.reset_index(drop=True)
ma_df.shape
ma_df.head()

,Unnamed: 0.1.1.1,Date,Year,Enforcement Action,Entity Name,Entity Type,Location,Text
0,0,8/29/2021,2003.0,Temporary Cease and Desist,Mortgage One Financial Corp,Mortgage Broker,Norwood MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
1,1,10/31/2021,2003.0,Temporary Cease and Desist,Platinum Capital Group,"Mortgage Lender,Mortgage Broker","HQ Irvine CA, branch located in Concord CA","COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
2,2,10/31/2021,2003.0,Temporary Cease and Desist,"KN Management, Inc.",Mortgage Broker,Concord MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
3,3,12/23/2021,2003.0,Temporary Cease and Desist,INSTAFI.COM,"Mortgage Lender,Mortgage Broker",Irvine CA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
4,4,12/30/2021,2003.0,Consent Order,"NovaStar Home Mortgage, Inc.","Mortgage Lender,Mortgage Broker",Kansas City MO,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."


In [8]:
ma_df = ma_df.loc[pd.notna(ma_df['Text'])]
ma_df.head()
#missing_df = missing_df[['Filename']]
#missing_df.to_csv("Ohio_missing.csv")

,Unnamed: 0.1.1.1,Date,Year,Enforcement Action,Entity Name,Entity Type,Location,Text
0,0,8/29/2021,2003.0,Temporary Cease and Desist,Mortgage One Financial Corp,Mortgage Broker,Norwood MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
1,1,10/31/2021,2003.0,Temporary Cease and Desist,Platinum Capital Group,"Mortgage Lender,Mortgage Broker","HQ Irvine CA, branch located in Concord CA","COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
2,2,10/31/2021,2003.0,Temporary Cease and Desist,"KN Management, Inc.",Mortgage Broker,Concord MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
3,3,12/23/2021,2003.0,Temporary Cease and Desist,INSTAFI.COM,"Mortgage Lender,Mortgage Broker",Irvine CA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
4,4,12/30/2021,2003.0,Consent Order,"NovaStar Home Mortgage, Inc.","Mortgage Lender,Mortgage Broker",Kansas City MO,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."


In [9]:
# Clean year for Massachusetts
ma_df['Year'] = ma_df['Year'].astype(str).replace(".0","")
ma_df['Year'] = ma_df['Year'].apply(lambda x: x.replace(".0",""))
ma_df['Date'] = ma_df.apply(lambda row: row['Date'].replace("2021",row['Year']), axis=1)
ma_df['Date'] = ma_df.apply(lambda row: row['Date'].replace("2029",row['Year']), axis=1)
ma_df.drop(['Year'], axis=1, inplace=True)
# Standardize Dates
ma_df['Date'] = ma_df.Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))
ohio_df['Date'] = ohio_df.Date.apply(lambda x: pd.to_datetime(x).strftime('%m/%d/%Y'))

In [10]:
# To upper all text in Massachusetts
ma_df['Enforcement Action'] = ma_df['Enforcement Action'].apply(lambda x: x.upper())
ma_df['Entity Name'] = ma_df['Entity Name'].apply(lambda x: x.upper())
ma_df['Entity Type'] = ma_df['Entity Type'].apply(lambda x: x.upper())
# To upper all text in Ohio as well
ohio_df['Credential'] = ohio_df['Credential'].apply(lambda x: x.upper())
ma_df.head(2)

,Unnamed: 0.1.1.1,Date,Enforcement Action,Entity Name,Entity Type,Location,Text
0,0,08/29/2003,TEMPORARY CEASE AND DESIST,MORTGAGE ONE FINANCIAL CORP,MORTGAGE BROKER,Norwood MA,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."
1,1,10/31/2003,TEMPORARY CEASE AND DESIST,PLATINUM CAPITAL GROUP,"MORTGAGE LENDER,MORTGAGE BROKER","HQ Irvine CA, branch located in Concord CA","COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ..."


In [11]:
# Standardize Headers
ma_df = ma_df.rename(columns={
    'Location':'City, State'
})
ohio_df = ohio_df.rename(columns={
    'Credential':'Entity Type',
    'Name': 'Entity Name',
    'Action': 'Enforcement Action'
})
# Adding State Label
ma_df['State Enforcing'] = 'MA'
ohio_df['State Enforcing'] = 'OH'

In [12]:
# Merge Dataframes
final_df = pd.concat([ohio_df,ma_df])
final_df

,Entity Type,Entity Name,"City, State",Enforcement Action,Date,Text,State Enforcing,Unnamed: 0.1.1.1
0,RM,"UNITED MORTGAGE GROUP, INC.","PARMA, OH",REPORT AND RECOMMENDATION,01/27/2000,NaN,OH,NaN
1,RM,"SUPERIOR HOME MORTGAGE, LLC","EUCLID, OH",NOTICE OF INTENT TO DENY,04/12/2000,NaN,OH,NaN
2,RM,"UNITED MORTGAGE GROUP, INC.","PARMA, OH",DIVISION ORDER,04/12/2000,NaN,OH,NaN
3,RM,"SUPERIOR HOME MORTGAGE, LLC","EUCLID, OH",REPORT AND RECOMMENDATION,11/02/2000,NaN,OH,NaN
4,RM,"T. P. O. T. T., INC.","COLUMBUS, OH",NOTICE OF INTENT TO REVOKE,04/18/2001,STATE OF OHIO\nDEPARTMENT OF COMMERCE\nDIVISIO...,OH,NaN
...,...,...,...,...,...,...,...,...
377,DEBT COLLECTOR,"A.C.B. AMERICAN, INC.",NaN,CONSENT ORDER,12/07/2010,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ...",MA,377.0
378,MORTGAGE COMPANY,"ADVANCED MORTGAGE SERVICES, LLC",NaN,CONSENT ORDER,12/13/2010,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ...",MA,378.0
379,MORTGAGE BROKER,"A-PLUS MORTGAGE, LLC DBA APM MORTGAGE",NaN,CONSENT ORDER,12/13/2010,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ...",MA,379.0
380,FOREIGN TRANSMITTAL AGENCY,"BRAZ TRANSFERS, INC.",NaN,CONSENT ORDER,12/22/2010,"COMMONWEALTH OF MASSACHUSETTS Suffolk, SS. ...",MA,380.0


In [13]:
# Remove newlines and spaces
def remove_newlines(text):
    if not pd.isna(text):
        text= re.sub(r"(\n( ?))+", "\n", text)
        text = re.sub(r"   +", "  ", text)
        text = text.replace("\n"," ")
    return text
final_df['Text'] = final_df['Text'].apply(remove_newlines)

# Calculate document length
def calc_length(text):
    if not pd.isna(text):
        return len(text)
    return 0
final_df['Text Length'] = final_df['Text'].apply(calc_length)

# Remove Stop Words
def clean(text):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(str(text))
    filtered_sentence = [w.lower() for w in word_tokens if not w.lower() in stop_words]
    return " ".join(filtered_sentence)

final_df['Clean Text'] = final_df['Text'].apply(clean)


KeyError: "['Unnamed: 0.1.1.1'] not found in axis"

In [21]:
final_df['Clean Text'] = final_df['Clean Text'].replace('nan', '', regex=True)

In [22]:
# Output Files
np.set_printoptions(threshold=sys.maxsize)
ohio_df.to_csv(r"Standardized Data\Ohio_completed.csv")
ma_df.to_csv(r"Standardized Data\MA_completed.csv")
final_df.to_csv(r"Standardized Data\Final.csv")